In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb
from sklearn import cross_validation, metrics
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from datetime import date

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/Train.csv', engine='python')
train.head()

In [ ]:
test = pd.read_csv('../input/Test.csv')
test.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.describe(include='O')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
test.info()

In [ ]:
target = train['Disbursed']
n_train = train.shape[0]
n_test = test.shape[0]
IDcol = 'ID'
train.drop(['LoggedIn', 'Disbursed'], axis=1, inplace=True)

In [ ]:
data = pd.concat((train, test)).reset_index(drop=True)
data.shape

In [ ]:
data.drop(['City'], axis=1, inplace=True)

In [ ]:
data['DOB'] = pd.to_datetime(data['DOB'])

In [ ]:
data['DOB'][0]

In [ ]:
def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
data['Age'] = data.apply(lambda row: age(row['DOB']), axis=1)

In [ ]:
data.drop(['DOB'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
import math
math.isnan(data['EMI_Loan_Submitted'][0])

In [ ]:
def missing_values(col, new_col):
    new_values = []
    d = data.shape[0]
    for i in range(d):
        if math.isnan(col[i]) == True:
            new_values.append(1)
        else:
            new_values.append(0)
    frame = pd.DataFrame(data = new_values,
                         columns = {new_col})
    return frame
EMI_Loan_Submitted_Missing = missing_values(data['EMI_Loan_Submitted'], 'EMI_Loan_Submitted_Missing')
Interest_Rate_Missing = missing_values(data['Interest_Rate'], 'Interest_Rate_Missing')
Loan_Amount_Submitted_Missing = missing_values(data['Loan_Amount_Submitted'], 'Loan_Amount_Submitted_Missing')
Loan_Tenure_Submitted_Missing = missing_values(data['Loan_Tenure_Submitted'], 'Loan_Tenure_Submitted_Missing')
Processing_Fee_Missing = missing_values(data['Processing_Fee'], 'Processing_Fee_Missing')

In [ ]:
data = pd.concat((data, EMI_Loan_Submitted_Missing, Interest_Rate_Missing, Loan_Amount_Submitted_Missing, Loan_Tenure_Submitted_Missing, Processing_Fee_Missing), axis=1)

In [ ]:
data.drop(['EMI_Loan_Submitted', 'Interest_Rate', 'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'Processing_Fee',
           'Employer_Name', 'Lead_Creation_Date', 'Salary_Account'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
sns.countplot('Device_Type', data=data)

In [ ]:
plt.figure(figsize=(16, 9))
sns.countplot('Source', data=data)
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.isnull().sum().sort_values(ascending=False)

In [ ]:
data['Loan_Amount_Applied'].median()

In [ ]:
cols = ['Loan_Amount_Applied', 'Loan_Tenure_Applied', 'Existing_EMI']
for col in cols:
    data[col] = data[col].fillna(data[col].median())

In [ ]:
data.isnull().sum().max()

In [ ]:
data['Source'] = data['Source'].apply(lambda x: 'others' if x not in ['S122', 'S133'] else x)

In [ ]:
sns.countplot('Var1', data=data)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
cols = ['Gender', 'Mobile_Verified', 'Filled_Form', 'Device_Type', 'Var2', 'Var1', 'Source']
for col in cols:
    data[col] = label.fit(data[col]).transform(data[col])

In [ ]:
data = pd.get_dummies(data, columns=cols)
data.columns

In [ ]:
data.head()

In [ ]:
train = data[:n_train]
test = data[n_train:]

In [ ]:
train = pd.concat((train, target), axis=1)

In [ ]:
train.head()

In [ ]:
train.to_csv('train_modified.csv', index=False)
test.to_csv('test_modified.csv', index=False)